### Import libraries

#### Run the same model and add the correctly labeled tweets based on the prediction on the second model
- We used KNN to predict the second model. Out of the 75 test data, 47 were correct ~ 62%.

In [2]:
import numpy as np
import pandas as pd
import scipy as sp   
from sklearn import metrics
from textblob import TextBlob, Word
from nltk.stem.snowball import SnowballStemmer
%matplotlib inline

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

from sklearn.cross_validation import train_test_split #how does this work?
from sklearn.model_selection import train_test_split as tts

from sklearn.ensemble import GradientBoostingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier , VotingClassifier
from sklearn.neighbors import KNeighborsClassifier
neighbors = 5
from sklearn.naive_bayes import MultinomialNB
from nltk.classify import NaiveBayesClassifier  #how does this work?

from sklearn.model_selection import cross_val_score as cvs

/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [5]:
kanye2 = pd.read_csv('kanyewest.csv')
kanye2.head()

,text,label
0,RT @WSHHRAP: Rappers with the most Billboard H...,1.0
1,"Kanye and Cudi merge, together they rule as Kanye",1.0
2,RT @RappersSaid: Rappers with the most Billboa...,1.0
3,RT @c_drew_: Kanye shaped like he played barit...,1.0
4,RT @Bubbabue: @kanyewest Your album #ye is an ...,1.0


In [42]:
df = kanye2
df.to_csv('kanyewest.csv')

In [6]:
kanye2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2064 entries, 0 to 2063
Data columns (total 2 columns):
text     2064 non-null object
label    439 non-null float64
dtypes: float64(1), object(1)
memory usage: 32.3+ KB


In [7]:
df = kanye2.sort_values(by = 'label', ascending=True)
df1 = df.reset_index()
df1[438:].head()

,index,text,label
438,438,RT @thefader: .@donaldglover on Season 3 of At...,1.0
439,439,I need someone who can edit to put Bebe Zahara...,NaN
440,440,@BHOPDaMaestro by Lil Baby ft. Drake &amp; Kan...,NaN
441,441,I’ve never heard Kanye West talk until celebri...,NaN
442,442,Someone stop this guy https://t.co/bEXdtJViQR,NaN


In [8]:
kanye2train = df1[(df1['label'] ==0)|(df1['label']==1)]
kanye2train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 439 entries, 0 to 438
Data columns (total 3 columns):
index    439 non-null int64
text     439 non-null object
label    439 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 13.7+ KB


In [9]:
kanye2train.head()

,index,text,label
0,219,RT @taydrug: Taylor was dragged for saying she...,0.0
1,154,I 2nd this. RT @de_nice89: The edit button is ...,0.0
2,155,RT @taydrug: Taylor was dragged for saying she...,0.0
3,156,RT @taydrug: Taylor was dragged for saying she...,0.0
4,157,RT @Parkour_Lewis: Cause he slavery shamed our...,0.0


In [12]:
kanye2test = df1[df1['label'].isnull()][75:150]
kanye2test.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 75 entries, 514 to 588
Data columns (total 3 columns):
index    75 non-null int64
text     75 non-null object
label    0 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 2.3+ KB


In [13]:
kanye2test.head()

,index,text,label
514,514,RT @CornellWBrooks: “Kanye’s comment condemns ...,NaN
515,515,"""Kim Kardashian asks Twitter boss to add edit ...",NaN
516,516,RT @KINGPR1NCESS: Los Angeles huh https://t.co...,NaN
517,517,RT @OliverMcGee: Thx U 🙏 @KimKardashian 4 your...,NaN
518,518,RT @AnUgIyNigga: Drake literally wouldnt exist...,NaN


In [14]:
text = kanye2train['text']
print(type(text))
text.head()

<class 'pandas.core.series.Series'>


0    RT @taydrug: Taylor was dragged for saying she...
1    I 2nd this. RT @de_nice89: The edit button is ...
2    RT @taydrug: Taylor was dragged for saying she...
3    RT @taydrug: Taylor was dragged for saying she...
4    RT @Parkour_Lewis: Cause he slavery shamed our...
Name: text, dtype: object

In [15]:
toadd = kanye2test['text']
toadd

514    RT @CornellWBrooks: “Kanye’s comment condemns ...
515    "Kim Kardashian asks Twitter boss to add edit ...
516    RT @KINGPR1NCESS: Los Angeles huh https://t.co...
517    RT @OliverMcGee: Thx U 🙏 @KimKardashian 4 your...
518    RT @AnUgIyNigga: Drake literally wouldnt exist...
519    RT @WSHHRAP: It's now confirmed. Drake wrote t...
520    Jesus, I listened to Ye last night.\n\nKanye W...
521    For Kanye I'd go with "Guess Who's Back" but c...
522    When Kanye said: "How could you be so DR.Evil ...
523    Good piece on Owens BS @willsommer \n\nShe's a...
524                        Kanye West says “expresso” Rt
525    @TroyKeywon @dsmoothe32 @bomani_jones I still ...
526    The "Christian" Kanye is the best Kanye. His m...
527    Wana model for Kanye?\nModels: yeezy? Hell ya ...
528    @kanyewest don’t let this person bring down yo...
529    @redsteeze Somewhere Kanye is calling Kevin an...
530    This person is an opportunistic performance ar...
531    10 Things Kanye West Wan

### Let's try TF-IDF

In [16]:
tfidf = text.append(toadd, ignore_index = True)
tfidf.head()

0    RT @taydrug: Taylor was dragged for saying she...
1    I 2nd this. RT @de_nice89: The edit button is ...
2    RT @taydrug: Taylor was dragged for saying she...
3    RT @taydrug: Taylor was dragged for saying she...
4    RT @Parkour_Lewis: Cause he slavery shamed our...
Name: text, dtype: object

In [17]:
vect = TfidfVectorizer(stop_words='english')

In [18]:
tf = pd.DataFrame(vect.fit_transform(tfidf).toarray(), columns=vect.get_feature_names())

In [19]:
tf.head()

,0jujild7px,10,100,1063atl,10s,11,12,135,148lqlf5ao,16,...,ynxi6p3c7u,yo,yodeling,youtube,ywltohwavw,yxotobksfb,zmh9qhh6xk,znu3q5mi58,zth5gym7xn,zuqwdrzfdd
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [20]:
tf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 514 entries, 0 to 513
Columns: 1230 entries, 0jujild7px to zuqwdrzfdd
dtypes: float64(1230)
memory usage: 4.8 MB


In [21]:
reset = kanye2train['label'].reset_index()
del reset ['index']
reset.head()

,label
0,0.0
1,0.0
2,0.0
3,0.0
4,0.0


In [23]:
kanye = pd.concat([tf,reset], axis = 1,ignore_index= False)
kanye['label'][438:].head()

438    1.0
439    NaN
440    NaN
441    NaN
442    NaN
Name: label, dtype: float64

In [24]:
edittrain = kanye[:439]
edittest = kanye[439:]
droplabel = edittest.drop('label', axis = 1)

In [25]:
edittrain.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 439 entries, 0 to 438
Columns: 1231 entries, 0jujild7px to label
dtypes: float64(1231)
memory usage: 4.1 MB


In [26]:
edittest.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75 entries, 439 to 513
Columns: 1231 entries, 0jujild7px to label
dtypes: float64(1231)
memory usage: 721.4 KB


In [27]:
x = edittrain.drop('label', axis = 1)
y = edittrain['label']

In [28]:
xtrain, xtest, ytrain, ytest = tts(x,y, random_state = 0)

In [29]:
l = LogisticRegression()
d = DecisionTreeClassifier()
r = RandomForestClassifier()
g = GradientBoostingClassifier()
k = KNeighborsClassifier()
nb = MultinomialNB()
vc = VotingClassifier(estimators = [('l',l),('d',d),('r',r),('g',g),('k',k),('nb',nb)],
                      voting = 'hard', n_jobs = 5)

In [30]:
def classifier(ye):
    print(ye)
    clf = ye.fit(xtrain,ytrain)
    clf.predict(droplabel)
    print('Accuracy of train: ', ye.score(xtrain,ytrain))
    print('Accuracy of test: ', ye.score(xtest,ytest))
    score = cvs(ye,xtest,ytest,cv=5, scoring='accuracy')
    print('cross val score: ', score.mean())

In [31]:
classifier(LogisticRegression())

LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)
Accuracy of train:  0.9209726443768997
Accuracy of test:  0.8727272727272727
cross val score:  0.8536043666478449


In [32]:
classifier(DecisionTreeClassifier())

DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
            max_features=None, max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best')
Accuracy of train:  1.0
Accuracy of test:  0.8454545454545455
cross val score:  0.8908338038772822


In [33]:
classifier(RandomForestClassifier())

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=None, verbose=0,
            warm_start=False)
Accuracy of train:  0.9878419452887538
Accuracy of test:  0.8727272727272727
cross val score:  0.8540372670807452


In [34]:
classifier(GradientBoostingClassifier())

GradientBoostingClassifier(criterion='friedman_mse', init=None,
              learning_rate=0.1, loss='deviance', max_depth=3,
              max_features=None, max_leaf_nodes=None,
              min_impurity_decrease=0.0, min_impurity_split=None,
              min_samples_leaf=1, min_samples_split=2,
              min_weight_fraction_leaf=0.0, n_estimators=100,
              presort='auto', random_state=None, subsample=1.0, verbose=0,
              warm_start=False)
Accuracy of train:  1.0
Accuracy of test:  0.8636363636363636
cross val score:  0.8544325239977413


In [35]:
classifier(KNeighborsClassifier())

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
           metric_params=None, n_jobs=1, n_neighbors=5, p=2,
           weights='uniform')
Accuracy of train:  0.9240121580547113
Accuracy of test:  0.9181818181818182
cross val score:  0.8185017880670055


In [36]:
classifier(MultinomialNB())

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)
Accuracy of train:  0.9817629179331308
Accuracy of test:  0.8636363636363636
cross val score:  0.8718238283455675


In [37]:
classifier(vc)

VotingClassifier(estimators=[('l', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)), ('d', DecisionT...
           weights='uniform')), ('nb', MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True))],
         flatten_transform=None, n_jobs=5, voting='hard', weights=None)


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


Accuracy of train:  0.9969604863221885
Accuracy of test:  0.8818181818181818


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site

cross val score:  0.8544325239977413


/Users/jedvillareal/anaconda3/envs/ipykernel_py3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


#### Extracting Ypred

In [38]:
mnb = nb.fit(xtrain,ytrain)
ypred = mnb.predict(droplabel)
ypred

array([1., 1., 1., 0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 0., 1., 0., 0.,
       0., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 0., 1., 1., 0., 1., 0., 1., 1., 1., 1.,
       1., 1., 1., 1., 1., 1., 1., 1., 0., 0., 1., 1., 0., 1., 0., 1., 1.,
       1., 1., 1., 1., 1., 1., 0.])

In [39]:
len(ypred)

75

In [40]:
len(toadd)

75

In [41]:
results = pd.DataFrame(columns=['text','label'])
results['text'] = toadd
results['label'] = ypred
results.to_csv('kanye.csv')